In [ ]:
# tensorflow 2.8.0 GUP사용, addons tfa사용
!pip install -q tensorflow==2.8.0 tensorflow-addons==0.16.1 numpy pandas \
    scikit-learn h5py Pillow keras-cv-attention-models

     |████████████████████████████████| 668.3 MB 17 kB/s 
     |████████████████████████████████| 1.1 MB 6.5 MB/s 
     |████████████████████████████████| 508 kB 77.3 MB/s 
     |████████████████████████████████| 462 kB 74.4 MB/s 


import

In [ ]:
import os #파일 경로 폴더를 처리
import glob #검색조건에 해당하는 파일명 가지고 온다
import math #연산
import random #랜덤
import collections #빈도를 알아낸다
import numpy as np
import pandas as pd
from PIL import Image #이미지 처리 pillow

from sklearn.model_selection import StratifiedKFold #imbalanceset의 train:validation 비를 감안하여 나누는 것

from sklearn.preprocessing import LabelEncoder #명목형을 숫자로
import tensorflow as tf 
print('tf:', tf.__version__)
import tensorflow_addons as tfa #최신버전 연산,손실,옵티마이저 등 계산
print('tfa:', tfa.__version__)
from keras_cv_attention_models import efficientnet #사용할 모델

tf: 2.8.0
tfa: 0.16.1


settings

In [ ]:
#args는 인수를 의미
class args:
    data_dir = os.path.join(os.path.expanduser('~'), 'solution') # Directory containing CSV files, directory는 folder의 상위개념으로 파일의 저장소를 directory로 지정한다, 초기 경로 구성 요소 ~에서 홈 디렉토리로 확장 ~의 경우 home의 환경변수로대체
    data_tfrec_dir = os.path.join(os.path.expanduser('~'), 'solution/tfrec') # 디랙토리가 tfrec를 포함
    data_tfrec_test  = os.path.join(data_tfrec_dir, 'test.tfrecord*') # test에 wildcard = 한번에 명령
    data_preds_dir   = 'preds'         # 예측을 저장
    seed             = 31542           # 시드설정으로 난수 예측
    mixed_precision  = 'mixed_float16' # 속도와 메모리에서 이(메모리 절반),tensorflow제공
    job              = 'train_test'    # 작업할 내용 = train test
    n_folds          = 5               # fold 수, StratifiedKFold에 쓰이는 것으로 fold=5는 4:1=train:test를 의미 이때 비율을 유지한다
    n_channels       = 3               # rgb
    dim              = 512             # 픽셀크기
    n_examples_train = 3422            # 전체 트레인 중 4/5
    n_epochs         = 200             # epoch가 커도 된다. early stop
    batch_size       = 32              # Batch size: 32 for 4x GPU, 64 for TPUv2
    lr               = 1e-4            # Learning rate
    weights          = 'imagenet21k'   # 여기서 weights를 가지고 옴
    aug_ratio        = 0.9             # 총 증강 수와 상관 없이 증강이 이미지가 출력될 확률
    n_aug            = -1              # train의 증강되는 숫자 -1은 모두
    n_tta            = 1               # test의 증강되는 숫자 1은 안함
    n_classes        = 88              # 88개 카테고리
    class_weight     = None            # 각 클래스에 지정하지 않음
    create_tfrecords = True            # 나중에 makedirs에 쓰이는데 다수의 디렉토리 설정함
   
    # exist_ok=True로 설정하면 파일이 없을 시에 생성 있으면 그냥 넘어감, 아래에 종종 나옴
    
# Check max folds, assert는 뒤조건이 아니면 error가 나버림
assert args.n_folds <= 10, 'Max 10 folds supported'

definitions

In [ ]:
    #환경변수는 시스템에서 지정해주는 변수를 의미, os.environ[]은 안의 변수값을 불러온다
    #환경변수는 컴퓨터 작동에서 필요한 값을 의미하므로 건들때 조심해야한다
    #동일한 양상으로 변수를 설정함 = set seed
    #아래에 _=seeder(args.seed+fold_id)를 쓰긴 함
def seeder(seed):
    """Set seed"""
    os.environ['PYTHONHASHSEED'] = str(seed)

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    return seed
    

    #hyperparameter(설정값)을 모두 list로 보겠다.
    #hyperparameter는 모델의 외적인 요소, 모델링 경험으로 찾아가는 것이다.lr와 같이 사용자가 설정
    #parameter는 머신러닝 인자이다. 최적화된 값을 목표로 한다.가중치, 결정계수, 벡터값과 같이
    #parameter는 함수에서 매개변수로 설정되는 값이라면 argument는 함수에 전달하는 인자이다.
    #vars는 객체의 dictionary를 반환 
    #dictionary는 자료형의 인덱스를 가질수 있는 list
    #딕셔너리에 대한 값을 리스트로 반환하되 설정을 넣음
    #def train_predict()에서 쓰이는데 일단 패스
def lsargs():
    """List all hyperparameters"""
    for a in sorted([a for a in vars(args) if '__' not in a]): 
        print('%-20s %s' % (a, vars(args)[a]))


#fold=5 -> train을 train:validation = 4:1로 5번 학습한다. 가장 작은 label이 5개이므로 6이상은 불가하다.
def create_cv_split(data_dir, n_splits=5):
    """Read .csv files and create 5-fold split"""

    #train은 공유폴더에 사진+csv로 존재한다.
    #test를 따로 받음, 엑셀로 정리된 파일 이미지는 별도인듯하다    
    train_df = pd.read_csv(os.path.join(data_dir, 'train_df.csv'))
    test_df = pd.read_csv(os.path.join(data_dir, 'test_df.csv'))

    #test label을 unknown으로 한다 
    test_df['label'] = 'unk'

    #train과test 파일이 동일 위치에 있어야 한다.
    train_df['image'] = data_dir + '/train/' + train_df['file_name']
    test_df['image']  = data_dir + '/test/' + test_df['file_name']

    #전처리에서 명목형 자료 데이터를 숫자로 치환 
    le = LabelEncoder()

    #wood-good과 같은 label을 숫자로 변환, 총 88개가 나올 것이다.
    train_df['label_int'] = le.fit_transform(train_df['label'])

    #값을 넣기 위해 열 추가 
    test_df['label_int'] = 0
    train_df['fold_id'] = 0
    test_df['fold_id'] = 0

    #데이터분할할건데 4:1로 무작위 추출을 33의 버전으로 나눈다
    #kfold=5인 이유 가장 적은 label의 값이 5이기 때문
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=33)

    # train과 train의 label의 values(값을 의미)를 비율분할해서 인덱스와 값을 가지고 온다. fold_id가 index
    #train과 val(validation)을 고정학습하면 val이 파라미터튜닝되어 과적합이 된다.
    #이를 해결하려고 교차검증하는데 모든 데이터를 활용하면서 과적합을 방지하고 시간이 k배로 늘어난다
    #교차검정이 앙상블도 관련이 있다.
    #하이퍼파라미터 튜닝기법 = lr, cost, 정규화파라미터, 가중치초기화 등등
    #sample(frac=1.0)은 데이터프레임에서 특정비율의 무작위 표본 추출한다.(0~1)
    #1은 모두 추출한다는 것이다 0.5는 4개중 2개 
    for fold_id, (train_index, val_index) in enumerate(kf.split(train_df, train_df['label'].values)):
        train_df.loc[train_df.index.isin(val_index), 'fold_id'] = fold_id
    train_df = train_df.sample(frac=1.0, random_state=34)
    return train_df, test_df


    #tfrecold는 tensorflow에서 자료구조형, 빠른 입출력, 데이터의 직렬화를 담당한다.
    #tfrecord는 파일정보와 label을 한번에 관리해서 빠르고 직관적이다.
    #이미지는 라벨이 따로 저장되어 읽기에 코드가 복잡하다.
    # 데이터프레임에 들어가는 파일로 class를 만들 예정이다.
    #이미지 정보와 id, 라벨과 id를 4가지를 읽어서 한번에 저장하는 것
class TFRecordProcessor(object):
    """Class to create TFRecord files using input from a DataFrame"""
        self.n_examples = 0
        
    # class지정은 여러대의 계산기를 한 컴퓨터에서 돌리는 것을 의미한다.
    #__init__(self)은 앞을 초기화 하고 실행한다.
    #반복문을 돌리기위해 비우는 느낌
    def __init__(self):

      # BytesList won't unpack a string from an EagerTensor.
      #isinstance는 value가 타입이 맞으면 true를 보낸다.
      #tf.constant(0)는 변수를 선언한다.0의 type가 int이므로
      #tf.train.bytestlist은 문자열 타입 데이터의 객체이다.
      #tf. int64는 bool, int, uint를 담당한다
      #tf. float는 float32, 64를 담당한다.     
      #어떤 데이터 형이 와도 그거 저장해서 각 데이터형으로 리턴하겠다.ex) bytes는 string/byte로 받아서 byte로 리턴
    def _bytes_feature(self, value):
        if isinstance(value, type(tf.constant(0))):
            value = value.numpy() 
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    def _int_feature(self, value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    def _float_feature(self, value):
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

    def _process_example(self, ind, A, B, C, D):
        self.n_examples += 1
        feature = collections.OrderedDict() #ordereddict데이터를 순서대로 받는 역할을 한다.
        feature['image_id'] = self._bytes_feature(A[ind].encode('utf-8'))
        feature['image'] =    self._bytes_feature(tf.io.read_file(B[ind]))
        feature['label_id'] = self._bytes_feature(C[ind].encode('utf-8'))
        feature['label'] =    self._int_feature(D[ind])

        #encoder는 10진수를 2진수로 암호화 decoder는 2진수를 10진수로 복원
        #uft-8은 가변 길이 유니코드 인코딩 4바이트이다. 널리쓰임
        #uft-16은 사용하는 것보다 더 많은 바이트를 소모한다.
        #tf.train.example은 {'string':tf.train.feature}의 매핑
        example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
        self._writer.write(example_proto.SerializeToString())

    #
    def write_tfrecords(self, A, B, C, D, n_shards=1, file_out='train.tfrecord'):
        n_examples_per_shard = A.shape[0] // n_shards
        n_examples_remainder = A.shape[0] %  n_shards
        self.n_examples = 0
        for shard in range(n_shards):
            self._writer = tf.io.TFRecordWriter('%s-%05d-of-%05d' % (file_out, shard, n_shards))
            start = shard * n_examples_per_shard
            if shard == (n_shards - 1):
                end = (shard + 1) * n_examples_per_shard + n_examples_remainder
            else:
                end = (shard + 1) * n_examples_per_shard
            print('Shard %d of %d: (%d examples)' % (shard, n_shards, (end - start)))
            for i in range(start, end):
                self._process_example(i, A, B, C, D)
                print(i, end='\r')
            self._writer.close()
        return self.n_examples
        

# TPU에 접근해서 학습을 하겠다.
def init_tpu(tpu=None):
    """ Seamlessly init any accelerator: CPU, GPU, multi-GPU, TPU
    """
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu=tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print('[init_tpu] Master:      ', tpu.master())
    except ValueError:
        strategy = tf.distribute.MirroredStrategy() #기존 모델의 코드를 조금만 바꾸어 분산 훈련 진행 여러 tpu로 분할
        print('[init_tpu] TPU was not found')
    print('[init_tpu] Num replicas:', strategy.num_replicas_in_sync)
    return strategy


    #deterministic 결정론적 어떤 입력이든 동일한 과정, 결과로
    #tta는 test time augmentation으로 test할때 ensemble 느낌, buffer_size는 이름이 들어가는 칸, file_glob는 파일 경로로 불러온다,cache 빨리 불러오는 것
    #이미지를 불러옴
    #ds.map key와 값을 저장하기 유용한 데이터 구조
    #classback은 인자로 넘겨지는 함수를 콜백하는 함수. 특정조건에 호출
    #ckpt는 모델 가중치 체크포인트 파일(가중치만 있다 구조없다) 메타정보로 재학습가능하나 무겁다.
    #그외 pb(가중치+구조), pbtxt(pb의 txt=모델구조)있다.
    #repeat는 한번 더 실행한다.
    #drop_remainder는 true일때 batch 남는거 버림, false는 돌림
    #속도를 빠르게 하기위해 파이프라인으로 데이터를 가지고 온다 prefatch
def init_tfdata(files_glob, deterministic=True, batch_size=32, auto=-1, 
                parse_example=None, aug=None, tta=None, norm=None, 
                repeat=False, buffer_size=None, cache=False, drop_remainder=False):
    """ Creates tf.data.TFRecordDataset with appropriate parameters."""

    options = tf.data.Options()
    options.experimental_deterministic = deterministic
    files = tf.data.Dataset.list_files(files_glob, 
        shuffle=not deterministic).with_options(options) #not deterministic이란 같은 입력이라도 매번 다른 결과
    print('N tfrec files:', len(files))
    #
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=auto)
    ds = ds.with_options(options)
    ds = ds.map(parse_example, num_parallel_calls=auto)
    #
    if aug:
        ds = ds.map(aug, num_parallel_calls=auto)
    if tta:
        ds = ds.map(tta, num_parallel_calls=auto)
    if norm:
        ds = ds.map(norm, num_parallel_calls=auto)
    if repeat:
        ds = ds.repeat()
    if buffer_size:
        ds = ds.shuffle(buffer_size=buffer_size, 
            reshuffle_each_iteration=True)
    ds = ds.batch(batch_size=batch_size, 
        drop_remainder=drop_remainder)
    ds = ds.prefetch(auto)
    if cache:
        ds = ds.cache()
    return ds

    #wildcard는 여러파일을 한번에 저장하기 위한 것, *은 생략을 의미하는 wildecard 문자다
    #파일 용량이 많으니 마지막것을 제외하고 삭제하는 것을 의미 (마지막 가중치만 저장)
class KeepLastCKPT(tf.keras.callbacks.Callback):
    """ Sort all ckpt files matching the wildcard 
    and remove all except last.
    """
 
    def __init__(self, wildcard):
        super(KeepLastCKPT, self).__init__() #상속을 받는다
        self.wildcard = wildcard

    #매번 epoch가 시작될 때 호출, epoch는 0으로 시작하는 현재 epoch의 index이다.
    def on_epoch_begin(self, epoch, logs=None):
        files = sorted(tf.io.gfile.glob(self.wildcard))
        if len(files):
            for file in files[:-1]:
                tf.io.gfile.remove(file)
            print('Kept ckpt: %s' % files[-1])
        else:
            print('No ckpt to keep')

    #훈련이 끝날 때 호출
    def on_train_end(self, logs=None):
        files = sorted(tf.io.gfile.glob(self.wildcard))
        if len(files):
            for file in files[:-1]:
                tf.io.gfile.remove(file)
            print('\nKept ckpt (final): %s' % files[-1])
        else:
            print('\nNo ckpt to keep (final)')

# 간단한 정보를 보여주는 description
feature_description = {
    'image':    tf.io.FixedLenFeature([], tf.string),
    'label':    tf.io.FixedLenFeature([], tf.int64),
}


#이미지를 불러온다.
#tf.cast
def parse_example(example_proto):
    """Parse example and return image in uint8 dtype"""
    d = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(d['image'], channels=args.n_channels, dct_method='INTEGER_ACCURATE')
    image = tf.image.resize(image, [args.dim, args.dim])
    image = tf.reshape(image, [args.dim, args.dim, args.n_channels])
    image = tf.cast(image, tf.uint8)
    label = tf.cast(d['label'], tf.int32)
    return image, label


# List of augmentation functions
aug_func_list = [
    # x90 positions of orig image
    lambda image, label: (image, label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*90 ), label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*180), label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*270), label),
    # x90 positions of flipped-left-right image
    lambda image, label: (tf.image.flip_left_right(image), label),
    lambda image, label: (tf.image.transpose(image), label),
    lambda image, label: (tf.image.flip_up_down(image), label),
    lambda image, label: (tfa.image.rotate(tf.image.flip_up_down(image), math.pi/180*90), label),
    # x45 positions of orig image without central crop, fill const 0 (black)
    lambda image, label: (tfa.image.rotate(image, math.pi/180*45 ), label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*135), label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*225), label),
    lambda image, label: (tfa.image.rotate(image, math.pi/180*315), label),
]

# -1 means all augs, 0 means only aug_func_list[0] which does nothing
if args.n_aug == -1:
    args.n_aug = len(aug_func_list)
aug_func_list = aug_func_list[:args.n_aug]

# Only functions used for aug may be used for tta. 
# Separate list allows amount of tta to be less or equal to amount of aug
if args.n_tta == -1:
    args.n_tta = len(aug_func_list)
tta_func_list = aug_func_list[:args.n_tta]


class Lambda:
    """
    Callable class used to create unified augmentation functions for `tf.switch_case`
    """
    def __init__(self, func, image, label):
        self._func = func
        self._image = image
        self._label = label
    def __call__(self):
        return self._func(self._image, self._label)


#minval<= <maxval 범위
#aug가 12 범위가 0~11로 같은 갯수를 가진다.
def aug(image, label):
    """
    Run single transformation per call with `aug_ratio` probability
    """
    if args.n_aug > 1:
        image, label = tf.switch_case(
            branch_index=tf.random.uniform(
                shape=[],       
                minval=0, 
                maxval=round((args.n_aug-1)/args.aug_ratio), dtype=tf.int32),
            branch_fns=[Lambda(func, image, label) for func in aug_func_list],
            default=Lambda(aug_func_list[0], image, label),)
    return image, label


def norm(image, label):
    """Normalization"""
    image = tf.image.resize(image, [args.dim, args.dim])
    image = tf.reshape(image, [args.dim, args.dim, args.n_channels])
    image = tf.cast(image, tf.float32)
    label = tf.cast(tf.one_hot(label, args.n_classes), tf.int32)
    image = image / 255.0
    return image, label


def init_model(print_summary=True):
    """Initialize a model"""
    model = tf.keras.Sequential([
        efficientnet.EfficientNetV2L(
            input_shape=(args.dim, args.dim, args.n_channels), 
            pretrained=args.weights, 
            num_classes=0),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(args.n_classes, activation='softmax')
    ], name='model')
    model.compile(optimizer=tf.keras.optimizers.Adam(args.lr), 
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=[tfa.metrics.F1Score(num_classes=args.n_classes, 
                                               average='macro', 
                                               name='f1')])
    if print_summary:
        model.summary()
    return model


#학습
#설정값을 보는 코드
def train_predict():
    """Run training and prediction in each of 5 folds"""
    lsargs()
    for fold_id in range(0, args.n_folds):
        print('\n*****')
        print('Fold:', fold_id)
        print('*****\n')
        #--------------------------------------------------------------------------
        print('Clear session...')
        tf.keras.backend.clear_session() #캐쉬를 지운다
        #--------------------------------------------------------------------------
        print('Set fold-specific seed...')
        _ = seeder(args.seed + fold_id)
        #--------------------------------------------------------------------------
        #정밀도와 속도의 trade-off, 부동소수점
        #mixed_precision은 10진수를 2진수로 바꾼다(부동소수점)
        #메모리를 빌려서 32->16 빠르게 메모리 덜 소모하며 학습한다.        
        if args.mixed_precision is not None:
            print('Init Mixed Precision:', args.mixed_precision)
            policy = tf.keras.mixed_precision.experimental.Policy(args.mixed_precision)
            tf.keras.mixed_precision.experimental.set_policy(policy)
 
        else:
            print('Using default precision:', tf.keras.backend.floatx())
        #--------------------------------------------------------------------------
        print('FULL BATCH SHAPE: %d x %d x %d x %d' % (args.batch_size,
                                                       args.dim,
                                                       args.dim,
                                                       args.n_channels))
        #--------------------------------------------------------------------------
        print('N aug (including original):', len(aug_func_list))
        print('N tta (including original):', len(tta_func_list))
        #--------------------------------------------------------------------------
        # Init TPU
        print('Init TPU')
        strategy = init_tpu()
        #--------------------------------------------------------------------------
        # 파일을 불러와서 tfrec에 넣는다
        # Globs
        all_fold_ids = np.array(range(args.n_folds))
        train_fold_ids = all_fold_ids[all_fold_ids != fold_id]
        train_glob = os.path.join(
            args.data_tfrec_dir, 
            ('fold.[' + '%d'*(args.n_folds-1) + '].tfrecord*') % tuple(train_fold_ids))
        val_glob   = os.path.join(args.data_tfrec_dir, 'fold.[%d].tfrecord*' % fold_id)
        print('TRAIN GLOB:', train_glob)
        print('VAL   GLOB:', val_glob)
        print('TEST  GLOB:', args.data_tfrec_test)
        #--------------------------------------------------------------------------
        #
        print('Init datasets')
        train_ds = init_tfdata(train_glob, 
                               deterministic=False,  
                               batch_size=args.batch_size, 
                               auto=-1,
                               parse_example=parse_example, 
                               aug=aug, 
                               norm=norm,
                               repeat=True,
                               buffer_size=256, 
                               cache=False)
        val_ds = init_tfdata(val_glob, 
                             deterministic=True,  
                             batch_size=args.batch_size * 2, 
                             auto=-1,
                             parse_example=parse_example,
                             norm=norm,
                             repeat=False,  
                             cache=True)
        #--------------------------------------------------------------------------
        print('Init model')
        with strategy.scope():
            model = init_model(print_summary=True)
        #--------------------------------------------------------------------------
        print('Init callbacks')
        call_ckpt = tf.keras.callbacks.ModelCheckpoint(
            'model-f%d-e{epoch:03d}-{val_loss:.4f}-{val_%s:.4f}.h5' % (fold_id, 'f1'),
            monitor='val_f1',
            save_best_only=True,
            save_weights_only=True,
            mode='max',
            verbose=1)
        call_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_f1', 
            factor=0.5, 
            patience=4, 
            min_delta=1e-4,
            min_lr=1e-8,
            verbose=1,
            mode='max')
        #early_stop을 지시, 불필요하게 gpu와 시간이 소비되는 것을 막는다.
        #8번안에 1e-4차이가 안나면 멈춘다
        call_early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_f1',
            patience=8,
            min_delta=1e-4,
            mode='max',
            verbose=1)
        call_keep_last = KeepLastCKPT(wildcard='model-f%d-e*.h5' % fold_id)
        #-------------------------------------------------------------------------- 
        if 'train' in args.job:
            print('Fit (fold %d)' % fold_id)
            h = model.fit(
                train_ds,
                steps_per_epoch=args.n_examples_train // args.batch_size,
                epochs=args.n_epochs,
                validation_data=val_ds,
                callbacks=[call_ckpt,
                           call_reduce_lr,
                           call_early_stop,
                           call_keep_last],
                class_weight=args.class_weight,
            )
        #--------------------------------------------------------------------------
        #마지막게 아니라 제일 좋은 것을 불러온다
        m = sorted(glob.glob('model-f%d*.h5' % fold_id))[-1]
        print('Load model (fold %d): %s' % (fold_id, m))
        model.load_weights(m)
        #--------------------------------------------------------------------------
        # Predict
        #--------------------------------------------------------------------------
        for tta_id in range(len(tta_func_list)):
            print('Init datasets for prediction (fold %d, tta %d)' % (fold_id, tta_id))
            test_ds = init_tfdata(args.data_tfrec_test, 
                                  deterministic=True,  
                                  batch_size=args.batch_size * 2, 
                                  auto=-1,
                                  parse_example=parse_example,
                                  tta=tta_func_list[tta_id],
                                  norm=norm,
                                  repeat=False,
                                  cache=True)
            #--------------------------------------------------------------------------
            #앙상블하기위해 제일 좋은것을 배열로 저장
            if 'test' in args.job:
                print('Predict TEST (fold %d, tta %d)' % (fold_id, tta_id))
                y_pred_test = model.predict(test_ds, verbose=1)
                np.save(os.path.join(args.data_preds_dir, 'y_pred_test_fold_%d_tta_%d.npy' % (fold_id, tta_id)), y_pred_test)
            #--------------------------------------------------------------------------
            

#여러개의 모델을 평균해서 가장 큰 것을 가져온다
def ensemble():
    """Compute ensemble, apply debiasing, and save submission"""
    files = sorted(glob.glob(os.path.join(args.data_dir, 'model_*', 'preds', '*.npy')))
    probas = []
    for file in files:
        probas.append(np.load(file))
    
    probas = np.mean(probas, axis=0)
    label_top_2 = np.argsort(probas, axis=1)[:, ::-1][:, :2]
    probas_top_2 = np.sort(probas, axis=1)[:, ::-1][:, :2]
    
    train_df = pd.read_csv(os.path.join(args.data_dir, 'train_df.csv'))
    le = LabelEncoder()
    le = le.fit(train_df['label'])
    
    label_inv = []
    for col in range(2):
        label_inv.append(le.inverse_transform(label_top_2[:, col]))
    
    subm_df = pd.read_csv(os.path.join(args.data_dir, 'sample_submission.csv'))
    subm_df['label'] = label_inv[0]
    subm_df['label_2nd'] = label_inv[1]
    subm_df['proba'] = probas_top_2[:, 0]
    
    
    #시행착오에 대한 방안
    # Debiasing: if "good"-class confidence is low - replace with "defective"-class
    def fn(row):
        if 'good' in row['label'] and row['proba'] < 0.70:
            return row['label_2nd']
        else:
            return row['label']
    subm_df['label'] = subm_df.apply(fn, axis=1)
    subm_df[['index', 'label']].to_csv(os.path.join(args.data_dir, 'submission.csv'), index=False)

create TFRecord dataset

In [ ]:
#tfrecord는 tensorflow의 저장 바이너리 데이터 포맷
#이미지를 읽을때 메타데이터와 라벨정보를 분리해서 가져오는데 코드가 어렵다
#메타데이터는 사진에서 날짜 이미지 파일 크기  등을 의미한다.
#이미지는 매던 인코딩/디코딩 작업을 해야한다.
if args.create_tfrecords:

    os.makedirs(args.data_tfrec_dir, exist_ok=True)

    train_df, test_df = create_cv_split(args.data_dir, n_splits=args.n_folds)
    
    tfrp = TFRecordProcessor()
    
    # Train
    for fold_id in range(len(train_df['fold_id'].unique())):
        print('Fold:', fold_id)
        n_written = tfrp.write_tfrecords(
            train_df[train_df['fold_id'] == fold_id]['file_name'].values,
            train_df[train_df['fold_id'] == fold_id]['image'].values,
            train_df[train_df['fold_id'] == fold_id]['label'].values,
            train_df[train_df['fold_id'] == fold_id]['label_int'].values,
            #
            n_shards=1, 
            file_out=os.path.join(args.data_tfrec_dir, 'fold.%d.tfrecord' % fold_id))
    
    # Test
    n_written = tfrp.write_tfrecords(
        test_df['file_name'].values,
        test_df['image'].values,
        test_df['label'].values,
        test_df['label_int'].values,
        #
        n_shards=1,
        file_out=os.path.join(args.data_tfrec_dir, 'test.tfrecord'))


Model-1: train and predict

In [ ]:
d = os.path.join(args.data_dir, 'model_1')
os.makedirs(d, exist_ok=True)
os.chdir(d)
os.makedirs(args.data_preds_dir, exist_ok=True)
print(os.getcwd())

train_predict()